In [5]:
from diagrams import Diagram, Cluster, Edge
from diagrams.digitalocean.compute import Droplet
from diagrams.digitalocean.database import DbaasPrimary
from diagrams.elastic.elasticsearch import Logstash
from diagrams.aws.compute import EC2


#with Diagram("Simple Diagram") as diag:
#    EC2("web")

In [6]:
import os
from langchain_openai.chat_models.azure import AzureChatOpenAI

openai_api_key = os.getenv('AZURE_OPENAI_API_KEY')
openai_api_base = os.getenv('AZURE_OPENAI_ENDPOINT')
openai_api_version = os.getenv('AZURE_OPENAI_API_VERSION')
embeddings_model = os.getenv('AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME')
gpt_model = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")

llm = AzureChatOpenAI(
                openai_api_version=openai_api_version,
                azure_deployment=gpt_model,
            )

In [28]:
import base64

def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')
    

img = encode_image("./Tire_demo/Tire-change.jpg")

In [33]:
def describe_images(image_path):

    image = encode_image(image_path)
    messages = {
        "role": "user",
        "content":[
            {"type":"text", "text": "Tell me what's in the image in a simple sentence"},
            {"type": "image_url", "image_url":{"url": f"data:image/jpeg;base64,{image}", "detail":"low"}},
        
        ]
    }

    return {"image_description": llm.invoke([messages]).content, "path": image_path}


In [34]:
describe_images("./Tire_demo/images.jpg")

{'image_description': 'A person is using a wrench to change a tire on a car.',
 'path': './Tire_demo/images.jpg'}

In [43]:
from langchain_core.prompts import PromptTemplate

def diagram_creator(images, text):
    "Uses Graphiz to generate smart diagrams based on text and images."

    prompt = """You're a smart assistant that writes python code.
Your job is to write a python code that uses Graphviz to generate a flow chart of the following text and images.
Analyze and understand the text very carefully, highlight the main components. Do no add more than 6 nodes.
Review the images descriptions and use the image to create the flow chart when you think is appropiate. 
Plasee add images to the flow chart. To find the images path, use the "path" property of the images dictionary below.
The flow chart must be great and self-explanatory.

Text: {text}

Images: {images}

Return just the snippet of code. Nothing else.
"""

    prompt_template = PromptTemplate.from_template(prompt)
    chain  = prompt_template | llm
    chain = chain.invoke({"text": text, "images": images})

    return chain.content

In [36]:
text = """
To change a tire, first, find a safe, level location and set up the wheel chocks. Next, loosen the lug nuts, jack up the car, and remove the flat tire. Then, mount the spare tire, lower the car, and tighten the lug nuts. Finally, fully lower the car and check the lug nuts. 
Here's a more detailed breakdown:
1. Safety First:
Find a safe location: Pull over to a safe and level spot, away from traffic if possible. 
Engage the parking brake: Turn on your hazard lights and apply the parking brake. 
Place wheel chocks: Place the wheel chocks on the wheels diagonally opposite the tire you're changing. 
2. Prepare the Vehicle:
Loosen the lug nuts: Using the lug wrench, loosen the lug nuts (but don't remove them yet). 
Locate the jack point: Consult your owner's manual or look for the designated jack points on your car. 
Position the jack: Place the jack securely under the designated jacking point. 
3. Remove the Flat Tire:
Raise the vehicle: Raise the car using the jack, enough to remove the flat tire.
Remove lug nuts: Completely remove the lug nuts.
Remove the flat tire: Carefully remove the flat tire, placing it aside safely. 
4. Install the Spare Tire:
Mount the spare tire: Align the spare tire with the lug bolts and push it onto the hub.
Replace lug nuts: Place the lug nuts on the wheel and tighten them by hand. 
5. Lower the Vehicle and Tighten:
Lower the car:
Slowly lower the car until the spare tire is on the ground, but the weight is not fully on the spare.
Tighten lug nuts:
Use the lug wrench to tighten the lug nuts in a star pattern, then fully lower the car and tighten the lug nuts again. 
6. Final Steps:
Check lug nuts:
After driving a short distance, check the lug nuts again to ensure they are tight. 
Consult a professional:
If you're not comfortable changing a tire, it's best to call for roadside assistance or a professional mechanic. 
"""

images = {'image_description': 'A person is using a wrench to change a tire on a car.',
 'path': './Tire_demo/images.jpg'}

In [44]:
diagram = diagram_creator(text, images)

In [45]:
print(diagram)

```python
import graphviz

dot = graphviz.Digraph(comment='Tire Changing Process')

dot.node('A', '1. Safety First\n- Find a safe location\n- Engage parking brake\n- Place wheel chocks')
dot.node('B', '2. Prepare the Vehicle\n- Loosen lug nuts\n- Locate the jack point\n- Position the jack')
dot.node('C', '3. Remove the Flat Tire\n- Raise the vehicle\n- Remove lug nuts\n- Remove flat tire')
dot.node('D', '4. Install Spare Tire\n- Mount spare tire\n- Replace lug nuts')
dot.node('E', '5. Lower Vehicle and Tighten\n- Lower car\n- Tighten lug nuts in star pattern')
dot.node('F', '6. Final Steps\n- Check lug nuts\n- Consult a professional')

dot.node('Image', 'Person changing tire', shape='rect', image='./Tire_demo/images.jpg', label='', or='none')

dot.edges(['AB', 'BC', 'CD', 'DE', 'EF'])
dot.attr(ranksep='0.5')

dot.render('tire_changing_flowchart', format='png', cleanup=True)
```


In [25]:
images

['Tire_demo\\AdobeStock_274756878_resize-269x300.jpg',
 'Tire_demo\\images (1).jpg',
 'Tire_demo\\images.jpg',
 'Tire_demo\\Tire-change.jpg']

In [7]:
from graphviz import Digraph

# Create a new directed graph
dot = Digraph()

# Add nodes (steps) to the graph
dot.node('A', 'Connect printer to computer and power source.')
dot.node('B', 'Go to My Devices (Windows) or Printers and Scanners (Mac).')
dot.node('C', 'Click Add Device and select your printer.')
dot.node('D', 'Print a test page to ensure your printer works.')
dot.node('E', label='image', image = "../api/assets/generated_images/2025-02-27-14-58-20/genai_img_9a12a22b-ccdf-4828-bfdd-9cac4971f474.jpg")
dot.node('F', label="image2", image = "../api/assets/generated_images/2025-02-27-15-29-10/genai_img_b4b09826-76bb-4cc4-a268-730cda29bac7.jpg")
dot.node('G', label="image3", image = "../api/assets/test_imgs/images.jpg")

# Add edges (arrows) to connect the steps
dot.edge('A', 'B', 'Connect')
dot.edge('B', 'C', 'Next')
dot.edge('C', 'D', 'Confirm')
dot.edge('D', 'E')
dot.edge('E', 'F')
dot.edge('F', 'G')



dot.render("printer set up", view=True)

'printer set up.pdf'